In [9]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
License(s): CC0-1.0




  0%|          | 0.00/228M [00:00<?, ?B/s]
  0%|          | 1.00M/228M [00:00<03:09, 1.25MB/s]
  1%|          | 2.00M/228M [00:00<01:33, 2.54MB/s]
  2%|▏         | 4.00M/228M [00:01<00:44, 5.24MB/s]
  3%|▎         | 6.00M/228M [00:01<00:31, 7.41MB/s]
  4%|▎         | 8.00M/228M [00:01<00:24, 9.44MB/s]
  4%|▍         | 10.0M/228M [00:01<00:20, 11.2MB/s]
  5%|▌         | 12.0M/228M [00:01<00:18, 12.3MB/s]
  6%|▌         | 14.0M/228M [00:01<00:16, 13.3MB/s]
  7%|▋         | 16.0M/228M [00:01<00:15, 13.9MB/s]
  8%|▊         | 18.0M/228M [00:02<00:15, 14.7MB/s]
  9%|▉         | 20.0M/228M [00:02<00:14, 15.1MB/s]
 10%|▉         | 22.0M/228M [00:02<00:14, 15.3MB/s]
 11%|█         | 24.0M/228M [00:02<00:13, 15.3MB/s]
 11%|█▏        | 26.0M/228M [00:02<00:13, 15.8MB/s]
 12%|█▏        | 28.0M/228M [00:02<00:13, 15.6MB/s]
 13%|█▎        | 30.0M/228M [00:02<00:13, 15.7MB/s]
 14%|█▍        | 32.0M/228M [00:02<00:12, 16.1MB/s]
 15%|█▍        | 34.0M/228M [00:03<00:12, 16.1MB/s]
 16%|█▌        | 36.

In [10]:
import zipfile
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pa
from scipy.linalg import svd, diagsvd
import pandas as pd

zip_file_path = 'the-movies-dataset.zip'

extract_to = 'data'

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

os.remove(zip_file_path)

In [20]:
df = pd.read_csv('data/ratings_small.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import svd
import altair as alt

avaliacoes = df[['userId', 'movieId', 'rating']]
matriz_avaliacoes = avaliacoes.pivot(index='userId', columns='movieId', values='rating').fillna(0)

def decompor_matriz_e_prever(matriz, linha, coluna, componentes_a_manter=145):

    U, sigma, VT = svd(matriz, full_matrices=False)
    
    sigma[componentes_a_manter:] = 0
    Sigma = np.diag(sigma)
    
    matriz_aproximada = np.dot(U, np.dot(Sigma, VT))
    
    return matriz_aproximada[linha, coluna]

def calcular_erros(matriz, num_experimentos=150, componentes_a_manter=145):
    
    erros = []

    while len(erros) < num_experimentos:

        linha = np.random.randint(0, matriz.shape[0])
        coluna = np.random.randint(0, matriz.shape[1])
        
        valor_original = matriz.iloc[linha, coluna]
        
        if valor_original == 0:
            continue 

        matriz_corrompida = matriz.copy()
        matriz_corrompida.iloc[linha, coluna] = np.random.uniform(0, 5)
        
        valor_previsto = decompor_matriz_e_prever(matriz_corrompida.values, linha, coluna, componentes_a_manter)

        erro = abs(valor_original - valor_previsto)
        erros.append(erro)

    return erros

def plotar_histograma_altair(erros):

    df_erros = pd.DataFrame({'Erro Absoluto': erros})
    
    histograma = alt.Chart(df_erros).mark_bar().encode(
        alt.X('Erro Absoluto:Q', bin=alt.Bin(maxbins=20), title='Erro Absoluto'),
        alt.Y('count()', title='Frequência'),
        tooltip=['count()']
    ).properties(
        title='Histograma dos Erros',
        width=600,
        height=400
    ).configure_axis(
        labelFontSize=12,
        titleFontSize=14
    ).configure_title(
        fontSize=16
    ).interactive()
    
    histograma.display()


erros_predicao = calcular_erros(matriz_avaliacoes, num_experimentos=100, componentes_a_manter=145)

plotar_histograma_altair(erros_predicao)


alt.Chart(...)